<a href="https://colab.research.google.com/github/Chaitu32/2D-game/blob/main/POIS_Task4_As1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Large n-bits prime generator
import random

small_primes_list = [ 2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97,
                     101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197,
                     199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313,
                     317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439,
                     443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571,
                     577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 673, 677, 683, 691,
                     701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809, 811, 821, 823, 827, 829,
                     839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 911, 919, 929, 937, 941, 947, 953, 967, 971, 977,
                     983, 991, 997]

def random_nbitnumber(n):
  return random.randrange(2**(n-1)+1,2**n -1)

def pseudo_prime_gen(n):

  while True:
    
    pseudo_PN = random_nbitnumber(n)
    #Testing for pseudo prime number

    prime_check = True

    for small_PM in small_primes_list:
      if pseudo_PN % small_PM == 0 and small_PM**2 <= pseudo_PN:
        prime_check = False
        break
    
    if prime_check : return pseudo_PN

def MillerRabinTest(pseudo_prime,trails):
  prev_num = pseudo_prime -1
  exp_2 = 0
  while prev_num %2 ==0:
    prev_num>>=1
    exp_2 +=1

  def check_composite(num_check):
    if pow(num_check,prev_num,pseudo_prime) == 1:
      return False
    for i in range(exp_2):
      if pow(num_check,2**i * prev_num,pseudo_prime) == pseudo_prime-1:
        return False
    return True
  
  for _ in range(trails):
    random_test = random.randrange(2,pseudo_prime)
    if check_composite(random_test): return False

  return True


def large_prime_gen(bits):
  prime = 0
  trails = 20
  while True:
    prime = pseudo_prime_gen(bits)
    
    if MillerRabinTest(prime,trails)== True:
      return prime

In [2]:
#@title Smallest primitive root finder
from math import sqrt
#primitive root finder
def primefactors(num):
  list_factors = set()

  while num %2 == 0 :
    list_factors.add(2)
    num >>=1

  for i in small_primes_list:
    while(num%i)==0:
      list_factors.add(i)
      num = num//i
  
  if num > max(small_primes_list)**2 :
    for i in range(max(small_primes_list)+2,int(sqrt(num)),2):
      while(num%i)==0:
        list_factors.add(i)
        num = num//i
  
  if (num>2):
    list_factors.add(num)

  return list_factors


def primitive_root(prime):
  prev_num = prime-1

  factors_num = primefactors(prev_num)

  for i in range(2,prime):
    check= True
    for j in factors_num:
      if(pow(i,prev_num//j,prime) ==1):
        check = False
        break
    if check :
      return i

  return -1

In [16]:
#@title Pseudo random number generator class
class PRNG():
  N = 64
  def __init__(self,output_bits,input_bits):
    if output_bits+input_bits < PRNG.N:
      self._init = True
      self._p = large_prime_gen(PRNG.N)
      print(self._p)
      self._g = primitive_root(self._p)
      if self._g == -1:
        print("No primitive roots for prime ",self._p)
        self._init = False
      print(self._g)
      self._output_bits = output_bits
      self._input_bits = input_bits
    else :
      self._init = False

  def __bool__(self):
    return self._init

  def generator(self,seed):
    if seed >= 2**self._input_bits:
      print("Invaild seed")
      return -1
    random_num = 0
    pow_2 = 1
    x_i = seed
    for _ in range(self._output_bits):
      x_i = pow(self._g,x_i,self._p)
      if x_i > (self._p-1)//2:
        random_num = random_num+pow_2
      pow_2 <<=1

    return random_num

In [17]:
#@title Pseudo random number function class

class PRNF():

  def __init__(self,input_bits):
    if input_bits*3 >= PRNG.N:
      print("Input bits are too large")
      return -1
    self._n = input_bits
    self._G = PRNG(2*input_bits,input_bits)

  def generator_function(self,key,x):
    if not (key < 2**self._n and x < 2**self._n):
      print("Invaild key or x")
      return -1

    random_num = 0
    for _ in range(self._n):
      x_i = x%2
      x >>=1
      random_num = self._G.generator(key)
      if x_i ==1:
        key = random_num>>self._n
      else :
        key = random_num%(2**self._n)

    return key


In [33]:
#@title Secure MAC
#Building secure MAC
import random

class MAC():

  def __init__(self,n):
    self._n = n
    self._F = PRNF(n)
    if self._F ==-1:
      print("Long input bits")
      return -1

  def Gen(self):
    return random.randrange(2**(self._n-1), 2**(self._n) -1)

  def MAC(self,key,m,r=None):
    if key >= 2**(self._n):
      print("Invaild key")
      return -1
    m_bits = 0
    while (1 << m_bits) < m:
      m_bits +=1
    if m_bits >= 2**(self._n//4 -1) :
      print("Message bits Invaild")
      return -1

    d = m_bits // (self._n//4)
    if d%(self._n/4) != 0:
      d +=1

    if r == None:
      r = random.randrange(2**(self._n//4-1),2**(self._n//4 ) - 1)
      #r = PRNG(self._n//4,self._n).generator(seed_r)
    

    tags = []
    t_bits = m_bits -1

    tag_inputs=[]
    for i in range(d):
      m_i = m >> t_bits
      m = m%(1<<t_bits)
      tag_input = m_i
      tag_input = (i<<(self._n//4)) + tag_input
      tag_input = (d<<(self._n//2)) + tag_input
      tag_input = (r<<(3* self._n//4)) + tag_input
      tag_inputs.append(tag_input)

    for tag_input in tag_inputs:
      tag = self._F.generator_function(key,tag_input)
      tags.append(tag)

    return [r]+tags


  def Vrfy(self,key,m,t):
    if len(t) ==0:
      print("Invaild tag")
      return -1
    output_tags = self.MAC(key,m,r=t[0])
    
    if len(output_tags)!=len(t):
      return False
    
    check = True
    for i,tag in enumerate(output_tags):
      if tag!=t[i] :
        check = False
        break

    return check 

In [34]:
mac = MAC(16)

12644425729206707573
2


In [35]:
key = mac.Gen()
key

60807

In [36]:
mac_t = mac.MAC(key,54)
mac_t

[8, 18553, 64749]

In [37]:
mac.Vrfy(key,54,mac_t)

True